In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import yfinance as yf
import datetime as dt

In [ ]:
ticker = ['SPY', 'TLT', 'META', 'NVDA', 'GLD', 'SNAP']
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days = 252)
interval = '1d'

In [ ]:
def getData(startDate, endDate, interval, ticker):
  tickerData = yf.download(ticker, startDate, endDate, interval=interval)
  tickerData = tickerData['Adj Close']
  tickerData = tickerData.pct_change().dropna()
  return tickerData

In [ ]:
pctData = getData(startDate, endDate, interval, ticker)
meanReturns = pctData.mean()
covMatrix = pctData.cov()

[*********************100%%**********************]  6 of 6 completed


In [ ]:
def portfolioPerformance(weights ,meanReturns, covMatrix):
  returns = np.sum(meanReturns*weights)*252
  std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights))) * np.sqrt(252)
  return returns, std

In [ ]:
def negativeSharpe(weights ,meanReturns, covMatrix):
  returns = np.sum(meanReturns*weights)*252
  std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights))) * np.sqrt(252)
  return -(returns/std)

In [ ]:
def portfolioVariance(weights, meanReturns, covMatrix):
  std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights))) * np.sqrt(252)
  return std

In [ ]:
numAseets = len(meanReturns)
bnds = []
b = (0.0, 1.0)
for i in range(numAseets):
  bnds.append(b)

In [ ]:
con1 = {'type':'eq', 'fun': lambda x: np.sum(x)-1}
cons = [con1]
x0 = [1./numAseets]*numAseets
args = (meanReturns, covMatrix)

In [ ]:
sol = minimize(negativeSharpe, x0, args, 'SLSQP', constraints=cons, bounds = bnds)

In [ ]:
sol.x.round(2), sol.fun.round(2)

(array([0.38, 0.32, 0.3 , 0.  , 0.  , 0.  ]), -4.15)

In [ ]:
for i in [i for i,v in enumerate(sol.x.round(2)) if v>0]:
  print(meanReturns.index[i])

GLD
META
NVDA


In [ ]:
portfolioPerformance(sol.x.round(2), meanReturns, covMatrix)

(1.0532313455962703, 0.2535340443297576)

In [ ]:
sol2 = minimize(portfolioVariance, x0, args,'SLSQP', constraints= cons, bounds=bnds)

In [ ]:
sol2.x.round(2), sol2.fun.round(4)

(array([0.4 , 0.  , 0.  , 0.  , 0.47, 0.13]), 0.0917)

In [ ]:
portfolioPerformance(sol2.x.round(2), meanReturns, covMatrix)

(0.1409110340420771, 0.0916988932211165)

In [ ]:
con2 = {'type': 'eq', 'fun': lambda x: portfolioPerformance(x, meanReturns, covMatrix )[0] - 0.30}
cons1 = [con1, con2]

In [ ]:
sol3 = minimize(portfolioVariance, x0, args, 'SLSQP', constraints=cons1, bounds=bnds)

In [ ]:
portfolioPerformance(sol3.x, meanReturns, covMatrix)